In [2]:
import torch



In [1]:
# lidar 2 tensor

import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import torch
import glob
import torch.nn.functional as F
from torchvision import transforms


# Class for the calibration matrices for KITTI data
class Calibration:
    def __init__(self, calib_filepath):
        calibs = self.read_calib_file(calib_filepath)

        self.P = calibs['P2']
        self.P = np.reshape(self.P, [3,4])

        self.L2C = calibs['Tr_velo_to_cam']
        self.L2C = np.reshape(self.L2C, [3,4])

        self.R0 = calibs['R0_rect']
        self.R0 = np.reshape(self.R0,[3,3])

    @staticmethod
    def read_calib_file(filepath):
        data = {}
        with open(filepath, 'r') as f:
            for line in f.readlines():
                line = line.rstrip()
                if len(line)==0: continue
                key, value = line.split(':', 1)
                try:
                    data[key] = np.array([float(x) for x in value.split()])
                except ValueError:
                    pass
        return data
    
    # From LiDAR coordinate system to Camera Coordinate system
    def lidar2cam(self, pts_3d_lidar):
        n = pts_3d_lidar.shape[0]
        pts_3d_hom = np.hstack((pts_3d_lidar, np.ones((n,1))))
        pts_3d_cam_ref = np.dot(pts_3d_hom, np.transpose(self.L2C))
        pts_3d_cam_rec = np.transpose(np.dot(self.R0, np.transpose(pts_3d_cam_ref)))
        return pts_3d_cam_rec
    
    # From Camera Coordinate system to Image frame
    def rect2Img(self, rect_pts, img_width, img_height):
        n = rect_pts.shape[0]
        points_hom = np.hstack((rect_pts, np.ones((n,1))))
        points_2d = np.dot(points_hom, np.transpose(self.P)) # nx3
        points_2d[:,0] /= points_2d[:,2]
        points_2d[:,1] /= points_2d[:,2]
        
        mask = (points_2d[:,0] >= 0) & (points_2d[:,0] <= img_width) & (points_2d[:,1] >= 0) & (points_2d[:,1] <= img_height)
        mask = mask & (rect_pts[:,2] > 2)
        return points_2d[mask,0:2], mask

# From Github https://github.com/balcilar/DenseDepthMap
def dense_map(Pts, n, m, grid): # something wrong!  
    # Actually there are a lot of errors in python script, because he forgets that the array begins in matlab with 1 but in python 0, which means array[0](in python)=array[1](in matlab). And if a range is (a:b), in matlab 'b' will be included and in python 'b' is excluded. 
    ng = 2 * grid + 1
    
    mX = np.zeros((m,n)) + float("inf")
    mY = np.zeros((m,n)) + float("inf")
    mD = np.zeros((m,n))
    mX[np.int32(Pts[1]),np.int32(Pts[0])] = Pts[0] - np.round(Pts[0])
    mY[np.int32(Pts[1]),np.int32(Pts[0])] = Pts[1] - np.round(Pts[1])
    mD[np.int32(Pts[1]),np.int32(Pts[0])] = Pts[2]
    
    KmX = np.zeros((ng, ng, m - ng, n - ng))
    KmY = np.zeros((ng, ng, m - ng, n - ng))
    KmD = np.zeros((ng, ng, m - ng, n - ng))
    
    for i in range(ng):
        for j in range(ng):
            KmX[i,j] = mX[i : (m - ng + i), j : (n - ng + j)] - grid - 1 +i
            KmY[i,j] = mY[i : (m - ng + i), j : (n - ng + j)] - grid - 1 +i
            KmD[i,j] = mD[i : (m - ng + i), j : (n - ng + j)]
    S = np.zeros_like(KmD[0,0])
    Y = np.zeros_like(KmD[0,0])
    
    for i in range(ng):
        for j in range(ng):
            s = 1/np.sqrt(KmX[i,j] * KmX[i,j] + KmY[i,j] * KmY[i,j])
            Y = Y + s * KmD[i,j]
            S = S + s
    
    S[S == 0] = 1
    out = np.zeros((m,n))
    out[grid + 1 : -grid, grid + 1 : -grid] = Y/S
    return out

import numpy as np

# From Github https://github.com/BerensRWU/DenseMap#readme
# From Github https://github.com/balcilar/DenseDepthMap

def dense_map_new(lidarOnImage, n, m, grid):
    # remove out of image size indexes
    mask = (lidarOnImage[:, :2] >= 1).all(axis=1)
    Pts = lidarOnImage[mask].T

    ng = 2 * grid + 1

    mX = np.zeros((m,n)) + float("inf") # inf-matrix
    mY = np.zeros((m,n)) + float("inf") # inf-matrix
    mD = np.zeros((m,n))
    mX[np.int32(np.round(Pts[1])-1),np.int32(np.round(Pts[0])-1)] = Pts[0] - np.round(Pts[0])
    mY[np.int32(np.round(Pts[1])-1),np.int32(np.round(Pts[0])-1)] = Pts[1] - np.round(Pts[1])
    mD[np.int32(np.round(Pts[1])-1),np.int32(np.round(Pts[0])-1)] = Pts[2]

    KmX = np.zeros((ng, ng, m - 2*grid, n - 2*grid))
    KmY = np.zeros((ng, ng, m - 2*grid, n - 2*grid))
    KmD = np.zeros((ng, ng, m - 2*grid, n - 2*grid))

    for i in range(ng):
        for j in range(ng):
            KmX[i,j] = mX[i : (m - 2*grid + i), j : (n - 2*grid + j)] - grid + j
            KmY[i,j] = mY[i : (m - 2*grid + i), j : (n - 2*grid + j)] - grid + i
            KmD[i,j] = mD[i : (m - 2*grid + i), j : (n - 2*grid + j)]

    S = np.zeros_like(KmD[0,0])
    Y = np.zeros_like(KmD[0,0])

    for i in range(ng):
        for j in range(ng):
            s = 1/np.sqrt(KmX[i,j] * KmX[i,j] + KmY[i,j] * KmY[i,j])
            Y = Y + s * KmD[i,j]
            S = S + s

    S[S == 0] = 1
    out = np.zeros((m,n))
    out[grid : -grid, grid  : -grid] = Y/S
    return out


if __name__ == "__main__":
    root = "../../public/dataset_expensive_storage/KITTI/testing/"
    image_dir = os.path.join(root, "image_2")
    velodyne_dir = os.path.join(root, "velodyne")
    calib_dir = os.path.join(root, "calib")

    tensors_max = torch.tensor(80, dtype=torch.float32)
    tensors_min = torch.tensor(0, dtype=torch.float32)
    
    depths_max_list = []
    length = 256
    
    # Data id
    for i in range(256,288):
        cur_id = i
        if i%10 ==0:
            print(f'{i}/{length}')
        # Loading the image
        img = cv2.imread(os.path.join(image_dir, "%06d.png" % cur_id))
        # Loading the LiDAR data
        lidar = np.fromfile(os.path.join(velodyne_dir, "%06d.bin" % cur_id), dtype=np.float32).reshape(-1, 4)
        # Loading Calibration
        calib = Calibration(os.path.join(calib_dir, "%06d.txt" % cur_id))
        # From LiDAR coordinate system to Camera Coordinate system
        lidar_rect = calib.lidar2cam(lidar[:,0:3])
        # From Camera Coordinate system to Image frame
        lidarOnImage, mask = calib.rect2Img(lidar_rect, img.shape[1], img.shape[0])
        # Concatenate LiDAR position with the intesity (3), with (2) we would have the depth
        lidarOnImage = np.concatenate((lidarOnImage, lidar_rect[mask,2].reshape(-1,1)), 1)

        # print(lidarOnImage.shape) --> (20285, 3)

        # 1. create depth map
        points = lidarOnImage.T
        # print(points.shape)
        u, v, depths = points

        convert_tensor = transforms.ToTensor()
        points = convert_tensor(points)
        # print(points.shape)

        depths = torch.tensor(depths)


        IMG_W = img.shape[1]
        IMG_H = img.shape[0]
        # print(IMG_H,IMG_W) --> 370 1224

        tensor_save = torch.zeros((1,IMG_H,IMG_W))

        dim0, dim1, dim2 = points.shape
        for i in range(dim2):
            y = points[0,0,i]
            y = y.long()
            x = points[0,1,i]
            x = x.long()
            z = depths[i]
            z = z.long()
            tensor_save[0,x,y] = z
        # print(tensor_save.shape)
        name = "%06d.pth" % cur_id
        tensor_save = tensor_save[0,150:350, 600:800].unsqueeze(0)
        # depth_max = torch.max(tensor_save)
        # depths_max_list.append(depth_max)

        tensor_save = (tensor_save-tensors_min)/(tensors_max-tensors_min)
        tensor_save = tensor_save.float()
        torch.save(tensor_save, f'testdataset/lidar_tensor/sparse/{name}')



    # print(max(depths_max_list))

#     print(tensor_save.shape)        



#         # 2. interpolation --> create dense depth map
#         out = dense_map_new(lidarOnImage, img.shape[1], img.shape[0], 6)
#         out_tensor = torch.unsqueeze(torch.from_numpy(out), dim=0)
#         # print(out_tensor.max())
#         out_tensor = out_tensor[0,150:350, 600:800].unsqueeze(0)

#         name = "%06d.pth" % cur_id

#         # resized_tensor = F.interpolate(out_tensor.unsqueeze(0), size=(100, 400), mode='bilinear', align_corners=False)
#         # resized_tensor = torch.squeeze(resized_tensor).unsqueeze(0)
#         # print(resized_tensor.max())

#         out_tensor = (out_tensor-tensors_min)/(tensors_max-tensors_min)
#         out_tensor = out_tensor.float()
#         # print(resized_tensor.shape)
#         torch.save(out_tensor, f'testdataset/lidar_tensor/dense/{name}')




260/256
270/256
280/256


In [2]:
# # A. mask / cut out

# import torch
# import glob
# import cv2
# import os
# import imgaug.augmenters as iaa

# # sample_token as name
# image_list = []
# ori_img_name_list = []
# damage_label = 'A'

# image_path = 'image_gray_200_original'
# images = sorted(glob.glob(image_path + '/*'))


# for i in range(len(images)):

#         image = cv2.imread(images[i])
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#         image_list.append(image)
#         ori_img_name = os.path.basename(images[i])
#         ori_img_name_list.append(ori_img_name)


# # print(len(image_list))
    
# seq = iaa.Sequential([
#     iaa.Cutout(nb_iterations=(4, 6), size=0.2, squared=False, random_state = 1 )
#     , ])

# images_aug_list = seq.augment_images(image_list)
# print(len(images_aug_list))

# for index in range(len(images_aug_list)):
#     # s = '%05d' % (index+1) # name format 00001
#     s = ori_img_name_list[index]
#     cv2.imwrite(f'testdataset/damaged_image/{damage_label}_{s}', images_aug_list[index])

# print('successful!')


32
successful!


In [1]:
# A. for later regionla mse

import numpy as np
import cv2
from matplotlib import pyplot as plt
import glob
import os

def create_mask(image_shape, block_size, num_blocks):
    mask = np.ones(image_shape, dtype=np.float32) 
    block_size_px = (int(image_shape[0] * block_size), int(image_shape[1] * block_size))
    np.random.seed(6)

    for _ in range(num_blocks):
        top_left = np.random.randint(0, image_shape[0] - block_size_px[0] + 1), np.random.randint(0, image_shape[1] - block_size_px[1] + 1)
        bottom_right = top_left[0] + block_size_px[0], top_left[1] + block_size_px[1]
        mask[top_left[0]:bottom_right[0], top_left[1]:bottom_right[1]] = 0
        
    return mask


image_list = []
ori_img_name_list = []
image_path = 'testdataset/ori_image'
images = sorted(glob.glob(image_path + '/*'))[0:32]
damage_label = 'A'

for i in range(len(images)):
    image = cv2.imread(images[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ori_img_name = os.path.basename(images[i])

    block_size = 0.2  # 方块大小为原图大小的百分比
    num_blocks = 16  # 方块数量
    mask = create_mask(image.shape[:2], block_size, num_blocks)
    damaged_img1 = image * mask
    cv2.imwrite(f'testdataset/damaged_image/{damage_label}_{ori_img_name}', damaged_img1)

    # cv2.imwrite(f'deepEmsemblesDataset/damaged_image_10/{ori_img_name}', damaged_img1)
    # print(image.shape[:2])

# # 显示损坏的图像，保留 mask 的效果
# plt.imshow(damaged_img1, cmap='gray')
# plt.show()


In [2]:
# B. motionblur

# loading library
import random
import glob
import cv2
import numpy as np
from pathlib import Path


def motion_blur(img):
    # Specify the kernel size.
    # The greater the size, the more the motion.
    kernel_size = random.randint(40, 70)

    # # Create the vertical kernel.
    kernel_v = np.zeros((kernel_size, kernel_size))

    # Create a copy of the same for creating the horizontal kernel.
    kernel_h = np.copy(kernel_v)

    # Fill the middle row with ones.
    # kernel_v[:, int((kernel_size - 1) / 2)] = np.ones(kernel_size)
    kernel_h[int((kernel_size - 1) / 2), :] = np.ones(kernel_size)

    # Normalize.
    # kernel_v /= kernel_size
    kernel_h /= kernel_size

    # # Apply the vertical kernel.
    # vertical_mb = cv2.filter2D(img, -1, kernel_v)

    # Apply the horizontal kernel.
    horizonal_mb = cv2.filter2D(img, -1, kernel_h)

    return horizonal_mb

    # Save the outputs.
    # cv2.imwrite('car_vertical.jpg', vertical_mb)
    
damage_label = 'B'
images_aug_list = []
ori_img_name_list = []

image_path = 'testdataset/ori_image'
images = sorted(glob.glob(image_path + '/*'))[32:64]

for i in range(len(images)):
    image = cv2.imread(images[i])
    image = motion_blur(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ori_img_name = os.path.basename(images[i])

    cv2.imwrite(f'testdataset/damaged_image/{damage_label}_{ori_img_name}', image)

print('successful!')


successful!


In [3]:
# C. sunflare
import torch
import os
import random
import glob
import cv2
import albumentations as A

damage_label = 'C'

image_path = 'testdataset/ori_image'
images = sorted(glob.glob(image_path + '/*'))[64:96]

print(len(images))
# fixed src center
# 一个list里面加的都一样！不采纳！
# flare_images= am.add_sun_flare(image, flare_center=(x,y), angle=-math.pi/4)
# hp.visualize(flare_images, column=3)

for index in range(len(images)):
    img = cv2.imread(images[index])

    transform = A.Compose(
         [A.RandomSunFlare(flare_roi=(0, 0, 1, 0.5), angle_lower=0.5, p=1, src_radius =200)],
    )
    random.seed()
    transformed = transform(image=img)
    img = transformed['image']
    image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ori_img_name = os.path.basename(images[index])
    cv2.imwrite(f'testdataset/damaged_image/{damage_label}_{ori_img_name}', image)


print('successful!')


32
successful!


In [4]:
# D. foggu

import glob
import cv2
import imgaug.augmenters as iaa

# sample_token as name
image_list = []
ori_img_name_list = []

damage_label = 'D'


image_path = 'testdataset/ori_image'
images = sorted(glob.glob(image_path + '/*'))[96:128]


# for i in range(len(images)):
#     image = cv2.imread(images[i])
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     ori_img_name = os.path.basename(images[i])
#     # cv2.imwrite(f'../../Dataset/lidar_img_16200/damaged_image_gray/D_fog/{ori_img_name}', image)
    
    
    
# for i in range(len(images)):

#         image = cv2.imread(images[i])
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#         image_list.append(image)
#         ori_img_name = os.path.basename(images[i])
#         ori_img_name_list.append(ori_img_name)


# seq = iaa.Sequential([
#     iaa.imgcorruptlike.Fog(severity=3),] )

# images_aug_list = seq.augment_images(image_list)

# for index in range(len(images_aug_list)):
#     s = ori_img_name_list[index]
#     cv2.imwrite(f'../../Dataset/lidar_img_16200/damaged_image_gray/D_fog/{s}', images_aug_list[index])


for i in range(len(images)):
    image = cv2.imread(images[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_list.append(image)


seq = iaa.Sequential([
    iaa.imgcorruptlike.Fog(severity=5),] )

images_aug_list = seq.augment_images(image_list)

for index in range(len(images_aug_list)):
    ori_img_name = os.path.basename(images[index])
    cv2.imwrite( f'testdataset/damaged_image/{damage_label}_{ori_img_name}', images_aug_list[index])

In [5]:
# E. overexposure

import cv2
import numpy as np
import random
import glob


def overexposure(img):
    # 将bgr转化为hsv
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img = img.astype(np.float64)
    # 获取v通道（颜色亮度通道），并做渐变性的增强
    Exp1 = random.randint(75, 90)
    Exp2 = random.randint(55, 65)
    Exp3 = random.randint(15, 25)

    img[:, :, 2] = np.where(img[:, :, 2] > 40, img[:, :, 2] + Exp1, img[:, :, 2])
    img[:, :, 2] = np.where(img[:, :, 2] > 140, img[:, :, 2] + Exp2, img[:, :, 2])
    img[:, :, 2] = np.where(img[:, :, 2] > 180, img[:, :, 2] + Exp3, img[:, :, 2])

    # 令大于255的像素值等于255（防止溢出）
    img = np.where(img > 255, 255, img)
    img = img.astype(np.uint8)
    res = cv2.cvtColor(img, cv2.COLOR_HSV2BGR)
    return res

images_aug_list = []
ori_img_name_list = []
damage_label = 'E'

image_path = 'testdataset/ori_image'
images = sorted(glob.glob(image_path + '/*'))[128:160]

print(len(images))
for i in range(len(images)):
    image = cv2.imread(images[i])
    image = overexposure(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ori_img_name = os.path.basename(images[i])
    # cv2.imwrite(f'../../Dataset/lidar_img_16200/damaged_image_gray/E_overexposure/{ori_img_name}', image)
    cv2.imwrite(f'testdataset/damaged_image/{damage_label}_{ori_img_name}', image)


32


In [6]:
# F. underexposure
import cv2
import numpy as np
import random
import glob

def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

image_path = 'testdataset/ori_image'
images = sorted(glob.glob(image_path + '/*'))[160:192]
damage_label = 'F'


for i in range(len(images)):
    image = cv2.imread(images[i])
    gamma = random.uniform(0.15, 0.3)
    image = adjust_gamma(image, gamma)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ori_img_name = os.path.basename(images[i])
    cv2.imwrite(f'testdataset/damaged_image/{damage_label}_{ori_img_name}', image)

In [7]:
# G condensed water
import torch
import glob
import cv2
import os
import imgaug.augmenters as iaa

image_list=[]
image_path = 'testdataset/ori_image'
images = sorted(glob.glob(image_path + '/*'))[192:224]
damage_label = 'G'

print(len(images))
for i in range(len(images)):
    image = cv2.imread(images[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_list.append(image)


seq = iaa.Sequential([
    iaa.imgcorruptlike.Spatter(severity=2),] )

images_aug_list = seq.augment_images(image_list)
print(len(images_aug_list))

for index in range(len(images_aug_list)):
    ori_img_name = os.path.basename(images[index])
    cv2.imwrite( f'testdataset/damaged_image/{damage_label}_{ori_img_name}', images_aug_list[index])


32
32


In [8]:
# H powder cluster

image_path = 'testdataset/ori_image'
image_list=[]
images = sorted(glob.glob(image_path + '/*'))[224:256]
damage_label = 'H'

for i in range(len(images)):
    image = cv2.imread(images[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_list.append(image)


seq = iaa.Sequential([
    iaa.imgcorruptlike.Spatter(severity=5),] )

images_aug_list = seq.augment_images(image_list)

for index in range(len(images_aug_list)):
    ori_img_name = os.path.basename(images[index])
    cv2.imwrite( f'testdataset/damaged_image/{damage_label}_{ori_img_name}', images_aug_list[index])





In [64]:
# I regionalPixelShift
import random
import glob
import cv2
import numpy as np

CAM_FRONT_path = '../../Dataset/lidar_img_16200/original_image_gray'
images = sorted(glob.glob(CAM_FRONT_path + '/*'))[14400:16200]

def pixel_shift(img):
    out = img.copy()
    out_mid = img.copy()
    out_mid2 = img.copy()
    shift1 = random.randint(5, 10)
    width1 = random.randint(12, 20)
    x1 = random.randint(0, 1600 - 3 * width1)

    out[:, x1 - 3 * width1:x1, :] = out_mid[:, x1:x1 + 3 * width1, :]
    out[:, x1:x1 + 3 * width1, :] = out_mid2[:, x1 - 3 * width1:x1, :]
    # out[x1 - shift1:x1 - shift1 + width1, :, :] = out[x1:x1 + width1, :, :]
    return out


for i in range(5):
    image = cv2.imread(images[i])
    image = pixel_shift(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ori_img_name = os.path.basename(images[i])
    cv2.imwrite(f'../../Dataset/lidar_img_16200/damaged_image_gray/I_regionalPixelShift/{ori_img_name}', image)

In [63]:
# test the geerated gray image
# Import necessary libraries
import torch
from PIL import Image
import torchvision.transforms as transforms
img=Image.open('../../Dataset/lidar_img_16200/damaged_image_gray/F_underexposure/8e9c6424a0e642dc97d75baf6d4c6d6f.png')

print(type(img))

transform = transforms.Compose([
    transforms.PILToTensor()
])

img_tensor = transform(img)
print(type(img_tensor))
print(img_tensor.shape)


<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'torch.Tensor'>
torch.Size([1, 256, 256])


In [1]:
# create only the masked dataset 
import torch
import glob
import cv2
import os
import imgaug.augmenters as iaa

# sample_token as name
image_list = []
ori_img_name_list = []

CAM_FRONT_path = '../../Dataset/lidar_img_16200/original_image_gray'
images = sorted(glob.glob(CAM_FRONT_path + '/*'))

for i in range(len(images)):

        image = cv2.imread(images[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image_list.append(image)
        ori_img_name = os.path.basename(images[i])
        ori_img_name_list.append(ori_img_name)


# print(len(image_list))
    
seq = iaa.Sequential([
    iaa.Cutout(nb_iterations=(16, 24), size=0.2, squared=False, seed =1 )
    , ])

images_aug_list = seq.augment_images(image_list)

for index in range(len(images_aug_list)):
    # s = '%05d' % (index+1) # name format 00001
    s = ori_img_name_list[index]
    cv2.imwrite(f'../../Dataset/lidar_img_16200/maksed_image_gray/{s}', images_aug_list[index])


In [13]:

images_aug_list = []
ori_img_name_list = []


CAM_FRONT_path = '../../Dataset/lidar_img_16200/original_image_gray'
# images = sorted(glob.glob(CAM_FRONT_path + '/*'))[1800:3600]
images = sorted(glob.glob(CAM_FRONT_path + '/*'))[14624:14656]
print(len(images))
for i in range(len(images)):
    image = cv2.imread(images[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ori_img_name = os.path.basename(images[i])
    cv2.imwrite(f'../../Dataset/lidar_img_16200/damaged_image_gray_all/for_porcessing/{ori_img_name}', image)

32
